In [1]:
%load_ext autoreload
%autoreload 2

from datasets.kaggle import KaggleDataset
import pandas as pd
import numpy as np

np.set_printoptions(precision=4, suppress=True)


In [2]:
dataset = KaggleDataset("Daily")

price, _ = dataset.get_price_date_series(pd.Timestamp("2014-1-1"), pd.Timestamp("2020-1-1"))
test_price, _ =  dataset.get_price_date_series(pd.Timestamp("2020-1-1"), pd.Timestamp("2030-1-1"))



In [3]:
from bot import sim_bot


def make_eval_fn(signal_fn, data):
    return lambda params: sim_bot(signal_fn(data, *params), data)

In [4]:
from signals import complex_bot_signal


make_eval_fn(complex_bot_signal, price)(
    # [4.278230671973636, 0.865252195291832, 0.0, 3, 7, 10, 0.31059751807297614, 0.0, 0.120024368485694, 3.2940466241589794, 18, 16, 11, 0.2092176436327862]
[10, np.float64(7.754061641966129), 0, 9, 10, 2, np.float64(0.9345906704777743), np.float64(7.985432086118452), np.float64(7.946909890121933), np.float64(6.595885997648705), 39, 20, 40, np.float64(0.10363120673044302)]
)

np.float64(15853.902822652713)

In [5]:
from algos.woa import WOA
from algos.pso import PSO
from algos.hill import HC

eval_fn = make_eval_fn(complex_bot_signal, price)

woa = PSO(
    eval_fn,
    [
        # (0.01, 1),
        # (0.01, 1),
        # (0.01, 1),
        # (2, len(test_price)/2),
        # (2, len(test_price)/2),
        # (2, len(test_price)/2),
        # (0.01, 1),
        # (0.01, 1),
        # (0.01, 1),
        # (0.01, 1),
        # (2, len(test_price)/2),
        # (2, len(test_price)/2),
        # (2, len(test_price)/2),
        # (0.01, 1),

        (0, 10),
        (0, 10),
        (0, 10),
        (1, 10),
        (1, 10),
        (1, 10),
        (0.01, 1),
        (0, 10),
        (0, 10),
        (0, 10),
        (11, 40),
        (11, 40),
        (11, 40),
        (0.01, 1),
    ],
    [3, 4, 5, 10, 11, 12],
)

woa.optimise()

print("best sol: ", woa.best_params)
print("best score: ", eval_fn(woa.best_params))

woa.plot()

best sol:  [ 7.9843  8.0119  0.7462 10.     10.      3.      1.      3.3896  7.6558
  0.     17.     40.     28.      0.8545]
best score:  16258.233489238688


In [6]:
eval_fn = make_eval_fn(complex_bot_signal, test_price)
print("test score: ", eval_fn(woa.best_params))

test score:  3613.732697850849


notice how the the window size is limited to half the test set length, if our wmas were able to use data from before the given dataset (which is appropriate as in reality all historical data is available) we could have a window size as large as half of the entire length of history.